In [1]:
from easynlp.utils import get_pretrain_model_path
from easynlp.appzoo import TextImageGenerationPredictor
from easynlp.core import PredictorManager
from easynlp.appzoo import TextImageGeneration
from PIL import Image
import base64
from io import BytesIO
import torch

No module named 'easy_predict'


In [2]:
user_defined_parameters = {'max_generated_num': 1}
pretrained_model_name_or_path = get_pretrain_model_path('/home/yubin/EasyNLP-alibaba/tmp/finetune_model_MUGE')
predictor = TextImageGenerationPredictor(model_dir=pretrained_model_name_or_path, model_cls=TextImageGeneration,
                                        user_defined_parameters=user_defined_parameters)

Working with z of shape (1, 256, 16, 16) = 65536 dimensions.
Restored from /home/yubin/EasyNLP-alibaba/tmp/finetune_model_MUGE


In [3]:
data = ["黄色秋冬针织衫童装儿童内搭上衣", "绿色秋冬针织衫童装儿童内搭上衣"]
inputs = [{'idx': idx, 'first_sequence': data[idx]} for idx in range(len(data))]
inputs

[{'idx': 0, 'first_sequence': '黄色秋冬针织衫童装儿童内搭上衣'},
 {'idx': 1, 'first_sequence': '绿色秋冬针织衫童装儿童内搭上衣'}]

In [4]:
model_inputs = predictor.preprocess(inputs)
model_inputs

{'idx': [0, 1],
 'input_ids': [array([24326, 22066, 21288, 17484, 23535, 21686, 22520, 21381, 22547,
         17420, 21381, 17463, 19406, 17061, 22516, 16384, 16384, 16384,
         16384, 16384, 16384, 16384, 16384, 16384, 16384, 16384, 16384,
         16384, 16384, 16384, 16384, 16384]),
  array([21728, 22066, 21288, 17484, 23535, 21686, 22520, 21381, 22547,
         17420, 21381, 17463, 19406, 17061, 22516, 16384, 16384, 16384,
         16384, 16384, 16384, 16384, 16384, 16384, 16384, 16384, 16384,
         16384, 16384, 16384, 16384, 16384])]}

In [5]:
results = predictor.predict_postprocess(model_inputs)
# results

tensor([[24326, 22066, 21288, 17484, 23535, 21686, 22520, 21381, 22547, 17420,
         21381, 17463, 19406, 17061, 22516, 16384, 16384, 16384, 16384, 16384,
         16384, 16384, 16384, 16384, 16384, 16384, 16384, 16384, 16384, 16384,
         16384, 16384],
        [21728, 22066, 21288, 17484, 23535, 21686, 22520, 21381, 22547, 17420,
         21381, 17463, 19406, 17061, 22516, 16384, 16384, 16384, 16384, 16384,
         16384, 16384, 16384, 16384, 16384, 16384, 16384, 16384, 16384, 16384,
         16384, 16384]], device='cuda:0') tensor([  2, 256,  16,  16])
tensor([[24326, 22066, 21288, 17484, 23535, 21686, 22520, 21381, 22547, 17420,
         21381, 17463, 19406, 17061, 22516, 16384, 16384, 16384, 16384, 16384,
         16384, 16384, 16384, 16384, 16384, 16384, 16384, 16384, 16384, 16384,
         16384, 16384],
        [21728, 22066, 21288, 17484, 23535, 21686, 22520, 21381, 22547, 17420,
         21381, 17463, 19406, 17061, 22516, 16384, 16384, 16384, 16384, 16384,
         163

In [6]:
len(results[0]['gen_imgbase64'])

1

In [8]:
len(results)

2

In [59]:
# input1 = torch.tensor([[24326, 22066, 21288, 17484, 23535, 21686, 22520, 21381, 22547, 17420,
#          21381, 17463, 19406, 17061, 22516, 16384, 16384, 16384, 16384, 16384,
#          16384, 16384, 16384, 16384, 16384, 16384, 16384, 16384, 16384, 16384,
#          16384, 16384],
#         [21728, 22066, 21288, 17484, 23535, 21686, 22520, 21381, 22547, 17420,
#          21381, 17463, 19406, 17061, 22516, 16384, 16384, 16384, 16384, 16384,
#          16384, 16384, 16384, 16384, 16384, 16384, 16384, 16384, 16384, 16384,
#          16384, 16384]], device='cuda:0') 
# input1.shape


torch.Size([2, 32])

In [60]:
# input2 = torch.tensor([  2, 256,  16,  16])
# input2.shape

torch.Size([4])

In [51]:
# results = predictor.postprocess(model_outputs)
# results

In [11]:
def base64_to_image(imgbase64_str):
    image = Image.open(BytesIO(base64.urlsafe_b64decode(imgbase64_str)))
    return image
for text, result in zip(data, results):
    imgbase64_str_list = result['gen_imgbase64']
    imgpath_list = []
    for base64_idx in range(len(imgbase64_str_list)):
        imgbase64_str = imgbase64_str_list[base64_idx]
        image = base64_to_image(imgbase64_str)
        imgpath = '{}_{}.png'.format(text, base64_idx)
        import os
        while os.path.exists(imgpath):
            base64_idx += 1
            imgpath = '{}_{}.png'.format(text, base64_idx)
        image.save(imgpath)
        imgpath_list.append(imgpath)
    print ('text: {}, save generated image: {}'.format(text, imgpath_list))

text: 黄色秋冬针织衫童装儿童内搭上衣, save generated image: ['黄色秋冬针织衫童装儿童内搭上衣_27.png']
text: 绿色秋冬针织衫童装儿童内搭上衣, save generated image: ['绿色秋冬针织衫童装儿童内搭上衣_14.png']


In [3]:
# Export the trained model to ONNX
dummy_input1 = torch.tensor([[24326, 22066, 21288, 17484, 23535, 21686, 22520, 21381, 22547, 17420,
         21381, 17463, 19406, 17061, 22516, 16384, 16384, 16384, 16384, 16384,
         16384, 16384, 16384, 16384, 16384, 16384, 16384, 16384, 16384, 16384,
         16384, 16384]], device='cuda:0') 
dummy_input2 = torch.tensor([  1, 256,  16,  16])

example_output = predictor.model(dummy_input1, dummy_input2)
print(example_output.shape)

torch.onnx.export(predictor.model, 
                (dummy_input1, dummy_input2),
                "output/text2image.onnx", 
                opset_version=11,
                input_names=["text_ids", "cshape"],
                output_names=["gen_imgs"],
                dynamic_axes={"text_ids": {0: "batch_size"},
                              "cshape": {0: "batch_size"},
                              "gen_imgs": {0: "batch_size"}
                })

torch.Size([1, 3, 256, 256])


/home/yubin/EasyNLP-alibaba/easynlp/modelzoo/models/artist/modeling_artist.py:145: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert t <= self.block_size, "Cannot forward, model block size is exhausted."
/home/yubin/miniconda3/envs/torch/lib/python3.8/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /tmp/pip-req-build-ex__3qls/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divid

KeyboardInterrupt: 

In [53]:
# from torch.autograd import Variable
# import torch
# # Export the trained model to ONNX
# dummy_input = Variable(torch.randn(1, 32)).long() # one black and white 28 x 28 picture will be the input to the model
# torch.onnx.export(predictor.model, dummy_input, "output/text2image.onnx")

TypeError: forward() missing 1 required positional argument: 'zshape'